In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

# input and output

In [10]:
###input
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter.csv'
target_smiles_path = '../../../Results/not_lipid/target_smiles_not_lipid_complex.pickle'
target_model_smiles_path = '../../../Results/not_lipid/yeast_met_not_lipid.pickle'
yeast8_metabolites_path = '../../../Data/model/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 50
num_process = 5
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_re/'
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/not_lipid/top{num}_{cut_off}_re/'

sce_gene_clean_ec = '../../../Data/Saccharomyces_cerevisiae_maxsep.csv'
DeepEC_path = '../../../Data/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/uniprotkb_organism_id_559292_2023_11_08.tsv'
yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
rxndb_to_model_with_target_success_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/rxndb_to_model_with_target_top50_0.3_success.csv'

###output
retrosys_smiles_calculate_similarity_file_path_all = f'../../../Data_retrosynthesis/not_lipid/retrosys_smiles_calculate_similarity_{cut_off}_re.json'

retrosys_smiles_calculate_similarity_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_retrosynthesis_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_retrosynthesis_path + f'RXNdb_all_top{num}_{cut_off}.json'
# rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}_all.csv'
rxndb_drop_path = cut_off_retrosynthesis_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
# rxndb_met_max_score_uptake_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_uptake.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
# rxndb_to_model_add_ec_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}_add_ec.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
# YMDB_success_met_smile_uptake_file = cut_off_path + f'YMDB_success_met_smile_uptake_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
# YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_uptake_top{num}_{cut_off}.pickle'
# rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
# rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'
# metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
# rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
# yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'

In [3]:
#check if the output folder exists
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(cut_off_retrosynthesis_path):
    os.makedirs(cut_off_retrosynthesis_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

# Calculate the similarity between metabolite and rule

In [4]:
yeast_total_smiles = process_yeast_smiles(target_model_smiles_path)
# yeast_total_smiles = [neutralize_charge(x) for x in yeast_total_smiles]
yeast_total_smiles = [x for x in yeast_total_smiles if Chem.MolFromSmiles(x)]
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')

total_smiles 3680
total_smiles 3679
total_smiles 3679
total_smiles 2222


21921it [00:01, 11251.45it/s]


8674


21921it [00:02, 8851.76it/s] 

12283


In [5]:
retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path_all)

# choose top num similar metabolites
retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

100%|██████████| 8674/8674 [13:02<00:00, 11.08it/s]


# retrosynthesis

In [6]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)

100%|██████████| 21921/21921 [01:13<00:00, 298.85it/s]


In [7]:
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 50/50 [00:09<00:00,  5.42it/s]


In [8]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)

100%|██████████| 21921/21921 [04:40<00:00, 78.27it/s]  


# reaction count:4374621
# remove reactions beyond yeast metabolites pool
# reaction count:92924

In [11]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=num_process)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 21921/21921 [00:04<00:00, 4431.99it/s]


rxndb: 4374621
rxndb_drop: 92924


In [12]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})

(92924, 9)


# 检查能否连接到模型 

In [13]:
#get all the metabolites in the rxndb
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)
#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
#calculate the similarity of the metabolites
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)

100%|██████████| 100/100 [00:00<00:00, 211.20it/s]


number of metabolites in RXNDB: 4708


100%|██████████| 4708/4708 [04:50<00:00, 16.22it/s]


In [14]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=7)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)

100%|██████████| 100/100 [00:03<00:00, 29.21it/s]


Iteration 1 - Current success count: 4013
Iteration 1 - Current success_rxndbid count: 66232


100%|██████████| 100/100 [00:03<00:00, 27.91it/s]


Iteration 2 - Current success count: 4254
Iteration 2 - Current success_rxndbid count: 70322


100%|██████████| 100/100 [00:03<00:00, 27.69it/s]


...

Iteration 5 - Current success count: 4305
Iteration 5 - Current success_rxndbid count: 72148


100%|██████████| 100/100 [00:03<00:00, 26.21it/s]


Iteration 6 - Current success count: 4312
Iteration 6 - Current success_rxndbid count: 72239


100%|██████████| 100/100 [00:03<00:00, 25.46it/s]


Iteration 7 - Current success count: 4312
Iteration 7 - Current success_rxndbid count: 72239
final success 4312
final success_rxndbid 72239


In [15]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)

success number: 215
fail number: 360


In [16]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path)

(72239, 10)
(72239, 10)
(72239, 5)
(72239, 5)


# remove orphan reactions
# reaction count:72239